# Claude CLI - Gestion des Sessions

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Debutant  
**Duree :** 25 min  
**Prerequis :** Notebook 01 complete

## Objectifs d'Apprentissage

- [ ] Comprendre le concept de sessions Claude
- [ ] Continuer une conversation avec `-c`
- [ ] Reprendre une session specifique
- [ ] Creer un fork de session
- [ ] Scripter des conversations multi-tours

---

## 1. Configuration

In [ ]:
import sys
import subprocess
sys.path.insert(0, 'helpers')

SIMULATION_MODE = False

from claude_cli import run_claude, verify_installation, print_response
import claude_cli
claude_cli.SIMULATION_MODE = SIMULATION_MODE

if verify_installation():
    print("Claude CLI pret")
else:
    print("Claude CLI non installe")

## 2. Concept de Sessions

Par defaut, chaque appel `claude -p` est **sans etat** : Claude ne se souvient pas des echanges precedents.

Les **sessions** permettent de maintenir un contexte conversationnel :

```
┌─────────────────────────────────────────────────────┐
│                    Session                          │
│  ┌───────┐   ┌───────┐   ┌───────┐   ┌───────┐    │
│  │ Msg 1 │ → │ Msg 2 │ → │ Msg 3 │ → │ Msg 4 │    │
│  └───────┘   └───────┘   └───────┘   └───────┘    │
│       ↓           ↓           ↓           ↓        │
│  Claude se souvient de tout le contexte            │
└─────────────────────────────────────────────────────┘
```

## 3. Continuer une Conversation (-c)

Le flag `-c` (continue) reprend la derniere conversation :

```bash
claude -p "Premiere question"    # Cree une nouvelle session
claude -c "Question de suivi"    # Continue la meme session
```

In [ ]:
# Message 1 : Etablir un contexte
stdout, stderr, code = run_claude(
    "Je vais te poser des questions sur Python. D'accord ?"
)
print("=== Message 1 ===")
print_response(stdout, stderr, code)

In [ ]:
# Message 2 : Continuer avec -c
# Note: En mode script, on utilise extra_args pour passer -c
result = subprocess.run(
    ["claude", "-c", "Quelle est la difference entre list et tuple ?"],
    capture_output=True,
    text=True,
    timeout=60
)

print("=== Message 2 (suite) ===")
print(result.stdout)

In [ ]:
# Message 3 : Encore une suite
result = subprocess.run(
    ["claude", "-c", "Et pour un dict vs un set ?"],
    capture_output=True,
    text=True,
    timeout=60
)

print("=== Message 3 (suite) ===")
print(result.stdout)

## 4. Sessions avec ID

Chaque session a un identifiant unique. Vous pouvez le specifier pour reprendre une session specifique :

```bash
# Voir les sessions disponibles
claude /sessions

# Reprendre une session par ID
claude --session-id abc123 -p "Suite de la conversation"
```

In [ ]:
# Lister les sessions recentes
result = subprocess.run(
    ["claude", "/sessions"],
    capture_output=True,
    text=True,
    timeout=30
)

print("=== Sessions Disponibles ===")
print(result.stdout if result.stdout else "Aucune session ou commande non supportee")

## 5. Fork de Session

Le **fork** permet de creer une branche a partir d'une session existante :

```
Session originale:  A → B → C
                         ↓
Fork:                    B → D → E  (nouvelle branche)
```

Utile pour explorer des alternatives sans perdre la conversation originale.

In [ ]:
# Creer une conversation de base
stdout, stderr, code = run_claude(
    "Je developpe une API REST en Python. Quel framework recommandes-tu ?"
)
print("=== Question initiale ===")
print_response(stdout, stderr, code)

In [ ]:
# Continuer normalement
result = subprocess.run(
    ["claude", "-c", "Quels sont les avantages de FastAPI ?"],
    capture_output=True,
    text=True,
    timeout=60
)
print("=== Suite normale ===")
print(result.stdout)

In [ ]:
# Fork pour explorer Flask a la place
# Note: --fork-session cree une nouvelle branche
result = subprocess.run(
    ["claude", "-c", "--fork-session", "Et si je choisis Flask a la place ?"],
    capture_output=True,
    text=True,
    timeout=60
)
print("=== Fork (alternative Flask) ===")
print(result.stdout if result.returncode == 0 else "Fork non supporte dans cette version")

## 6. Script Multi-Tours

Voici comment creer une conversation programmatique avec plusieurs echanges :

In [ ]:
def conversation_multi_tours(messages: list) -> list:
    """
    Execute une conversation multi-tours.
    
    Args:
        messages: Liste de prompts a envoyer en sequence.
        
    Returns:
        Liste des reponses.
    """
    responses = []
    
    for i, msg in enumerate(messages):
        if i == 0:
            # Premier message : nouvelle session
            cmd = ["claude", "-p", msg]
        else:
            # Messages suivants : continuer
            cmd = ["claude", "-c", msg]
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=60
        )
        
        responses.append({
            "question": msg,
            "response": result.stdout,
            "success": result.returncode == 0
        })
    
    return responses

In [ ]:
# Exemple de conversation
conversation = [
    "Je veux creer un jeu en Python. Par quoi commencer ?",
    "Quel est le framework le plus simple pour debuter ?",
    "Peux-tu me donner un exemple de code minimal ?"
]

# Executer (attention : consomme des tokens)
# Decommentez pour executer
# results = conversation_multi_tours(conversation)
# for r in results:
#     print(f"Q: {r['question'][:50]}...")
#     print(f"R: {r['response'][:200]}...\n")

## 7. Exercices Pratiques

In [ ]:
# EXERCICE 1 : Creez une conversation de 3 messages sur un sujet technique
# Message 1 : Posez une question generale
# Message 2 : Demandez des details avec -c
# Message 3 : Demandez un exemple avec -c

# Votre code ici


In [ ]:
# EXERCICE 2 : Utilisez la fonction conversation_multi_tours
# pour creer une conversation sur les design patterns

design_pattern_conv = [
    # Completez avec 3-4 messages
]

# Votre code ici


## 8. Bonnes Pratiques

### Quand utiliser les sessions ?

| Situation | Recommandation |
|-----------|----------------|
| Question isolee | `claude -p` simple |
| Exploration iterative | Sessions avec `-c` |
| Debug progressif | Sessions avec `-c` |
| Comparaison d'approches | Fork de session |
| Script automatise | Nouvelle session a chaque run |

### Conseils

1. **Limitez la longueur** : Les sessions tres longues consomment beaucoup de tokens
2. **Resumez** : Demandez a Claude de resumer la conversation si elle devient longue
3. **Nouvelle session** : En cas de changement de sujet majeur, creez une nouvelle session

## 9. Resume

Dans ce notebook, nous avons appris :

- **Sessions** : Maintiennent le contexte entre les echanges
- **`-c`** : Continue la derniere conversation
- **`--session-id`** : Reprend une session specifique
- **`--fork-session`** : Cree une branche alternative
- **Scripts multi-tours** : Automatiser des conversations

### Prochaine etape

Dans le notebook suivant, nous verrons comment **referencer des fichiers** avec les @-mentions.

-> [03-Claude-CLI-References.ipynb](03-Claude-CLI-References.ipynb)